In [1]:
import qutip, pickle, sys
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as opt 
import scipy.linalg as linalg
import time as time
import math, cmath

import a_quantum_stateology_lib as qstates
import b_quantum_geometries_lib as gij
import c_matrix_analysis_lib as matAnsys 
import d_spin_representations_lib as su2
import e.maxent_solver_lib.py as me

In [3]:
N=6; chain_type = "XX"; k_B = 1; temp = 1e+1; temp_ref = 1e+1

### One body local spin operators
spin_ops_list = su2.one_body_spin_ops(size = N)
identity_op = spin_ops_list[0][0]

### Heisenberg Hamiltonian
Hamiltonian_paras = [-1e-1, -2.5e-1, 0., .5e-1]
H_loc = su2.Heisenberg_Hamiltonian(op_list = spin_ops_list, chain_type = chain_type,
                                           size = N, Hamiltonian_paras = Hamiltonian_paras,
                                           closed_bcs = True, visualization = False)

### Generating classical observables

cl_ops, label_ops = su2.classical_ops(size = N, Hamiltonian = H_loc,
                                                op_list = spin_ops_list, centered_x_op = False)

## The Hamiltonian is added a perturbation term 
perturbation_strength = max(Hamiltonian_paras)/2

HH = H_loc + perturbation_strength * cl_ops["x_op"]

In [4]:
# Exact evolution

K0 = (.5 * cl_ops["n_oc_op"])
rho0 = matAnsys.safe_expm_and_normalize(K = K0, return_free_energy = False)[0]

In [5]:
subset_of_observables = {}; subset_label_ops = []
subset_of_observables["x_op_sq"] = cl_ops["p_op"]; subset_label_ops.append("x_op_sq")
subset_of_observables["n_oc_op"] = cl_ops["n_oc_op"]; subset_label_ops.append("n_oc_op")
subset_of_observables["n_oc_op_sq"] = cl_ops["n_oc_op"]**2; subset_label_ops.append("n_oc_op_sq")
subset_of_observables["p_op"] = cl_ops["p_op"]; subset_label_ops.append("p_op")

In [6]:
ts = np.linspace(1e-1, 4., 500)
exact_ev = qutip.mesolve(H = HH,
                         rho0 = rho0,
                         tlist = ts,
                         e_ops = subset_of_observables
                         )

In [9]:
A = {}
A["gato"] = "Fita"
exact_ev.append(A)

AttributeError: 'Result' object has no attribute 'append'

In [ ]:
def solve_exact(Hamiltonian, K0, timespan, return_evolved_rhos = True,
                                           return_qutip_res_obj = True):
    
    res = qutip.mesolve(H = Hamiltonian, rho0 = K0, tlist = timespan,
                        c_ops = None, e_ops = None)
    if return_evolved_rhos: 
        rho_at_timet = [matAnsys.safe_expm_and_normalize(K = Kt1, return_free_energy = False, 
                                                                  tol = 1e-5) 
                           for Kt1 in res.states]
    return res, rho_at_timet
    

In [ ]:
def quantum_dynamics(Hamiltonian, K0, timespan, depth, beta, HB_B0):
    print("*** 1. Starting at", datetime.now)
    results = {}
    print("*** 2. Processing initial state of the system")
    print("*** 3. Building Hierarchical Basis of depth l = ", depth)
    HBasis_local = matAnsys.build_Hierarchical_Basis(Hamiltonian = H,
                                                     seed_operator = HB_B0[-1], 
                                                     depth = depth,
                                                     tol = 1e-5, verbose = False)
    HBasis_local = HB_B0.pop() + HBasis_local
    print("*** 4. Processing Manifold Restricted Dynamics.\n     4A. Fetching categories of inner products.\n ") 
    print("        ###. Solving with Exact Geometry")
    res_exact, rhot = me.solve_exact(Hamiltonian = Hamiltonian,
                                     K0 = K0, 
                                     timespan = timespan,
                                     return_evolved_rhos = True,
                                     return_qutip_res_obj = True)
    K_at_timet = res_exact.states
    print("        ###. Projected Max-Ent solution with: ")
    print("                                            A. KMB-geometry")
    instantaneous_KMB_geometry = [gij.fetch_Kubo_inner_prod(sigma = rho) for rho in rhot]
    print("                                            B. instantaneous correlation geometry")
    instantaneous_corr_geometry = [gij.fetch_corr_scalar_prod(sigma = rho) for rho in rhot]
    print("     4B. Computing evolutions and Projections.\n ")
    print("        ###. 1. KMB Geometry")
    result["KMB"] = [gij.instantaneous_proj(Kt, orth_basis, kmb_sp, kmb_sp)
                     for Kt, kmb_sp in zip(K_at_timet, instantaneous_KMB_geometry)]
    
    
    print("        ###. 2. Hilbert Schmidt Geometry")
    sp = gij.fetch_HS_scalar_product()
    orth_basis = gij.orthogonalize_basis(basis, sp)
    result["HS"] = [gij.static_proj(Kt, orth_basis, sp, kmb_sp)
                    for Kt, kmb_sp in zip(K_at_timet, instantaneous_KMB_geometry)
                        ]
    
    print("        ###. 3. Stationary Correlation Geometry")
    sp = gij.fetch_corr_scalar_product(state)
    orth_basis = gij.orthogonalize_basis(basis, sp)
    result["corr (stationary)"] = [gij.static_proj(Kt, orth_basis, sp, kmb_sp)
                                   for Kt, kmb_sp in zip(K_at_timet, instantaneous_KMB_geometry)]
    
    print("        ###. 4. Instantaneous Correlation Geometry")
    
    result["corr (instataneous)"] = [gij.instaneous_proj(Kt, orth_basis, corr_sp, kmb_sp)
                                     for Kt, kmb_sp in zip(K_at_timet, instantaneous_KMB_geometry)]
    
    print("***. Test 1: Testing the Hermiticity of the Basis")
    

In [1]:
print("A\nB")

A
B


In [ ]:
def solve_MaxEnt_KMB(Hamiltonian, K0, timespan):